# Importing Packages and Reading in Data

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
from datetime import datetime
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')
from pathlib import Path
DATA_DIR = Path('.')/'..'/'data'
import pickle
def isNaN(num):
    return num != num

In [2]:
governor = pd.read_csv(DATA_DIR/'cleaned'/'Clean_Governor_Winners.csv').drop("Unnamed: 0", axis = 1)
governor["date"] = pd.to_datetime(governor["date"])
governor = governor.sort_values(by = "date", ascending = False).reset_index(drop=True)
governor["name"] = governor["name"].str.replace(r" sr "," ")
governor["name"] = governor["name"].str.replace(r" jr "," ")
governor["name"] = governor["name"].str.replace(r"^[a-z ]+ ","")
governor["district"] = np.int(0)

house = pd.read_csv(DATA_DIR/'cleaned'/'Clean_House_Winners.csv').drop("Unnamed: 0", axis = 1)
house["date"] = pd.to_datetime(house["date"])
house = house.sort_values(by = "date", ascending = False).reset_index(drop=True)
house["name"] = house["name"].str.replace(r" sr "," ")
house["name"] = house["name"].str.replace(r" jr "," ")
house["name"] = house["name"].str.replace(r"^[a-z ]+ ","")

senate = pd.read_csv(DATA_DIR/'cleaned'/'Clean_Senate_Winners.csv').drop("Unnamed: 0", axis = 1)
senate["date"] = pd.to_datetime(senate["date"])
senate = senate.sort_values(by = "date", ascending = False).reset_index(drop=True)
senate["name"] = senate["name"].str.replace(r" sr "," ")
senate["name"] = senate["name"].str.replace(r" jr "," ")
senate["name"] = senate["name"].str.replace(r"^[a-z ]+ ","")
senate["district"] = np.int(0)

nick = pd.read_pickle(DATA_DIR/'cleaned'/'poll_race_candidates.pkl')
nick["winner"] = "<BLANK>"
nick["winning_party"] = "<BLANK>"
nick["date"] = pd.to_datetime(nick["date"])
nick["district"] = np.int(0)
nick.loc[(nick['race_name'].str.contains(r"n\.h\.")) & ([isNaN(x) for x in nick["state"]]), "state"] = "NH" 


# Separating into relevant portions

In [3]:
df_s = nick[nick['race_name'].str.contains(r"[Ss]enate")].reset_index(drop=True)
df_s

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
0,florida senate,senate,FL,"[crist, rubio]",quinnipiac,"{'crist': 32.0, 'rubio': 30.0, 'meek': 24.0}",crist,2.0,2010-04-15,<BLANK>,<BLANK>,0
1,arkansas senate,senate,AR,"[boozman, lincoln]",r2000/daily kos (d),"{'boozman': 50.0, 'lincoln': 43.0}",boozman,7.0,2010-04-15,<BLANK>,<BLANK>,0
2,arkansas senate,senate,AR,"[boozman, halter]",r2000/daily kos (d),"{'boozman': 48.0, 'halter': 41.0}",boozman,7.0,2010-04-15,<BLANK>,<BLANK>,0
3,florida senate,senate,FL,"[rubio, meek]",quinnipiac,"{'rubio': 42.0, 'meek': 38.0}",rubio,4.0,2010-04-15,<BLANK>,<BLANK>,0
4,florida senate,senate,FL,"[rubio, crist]",quinnipiac,"{'rubio': 56.0, 'crist': 33.0}",rubio,23.0,2010-04-15,<BLANK>,<BLANK>,0
5,arkansas senate,senate,AR,"[lincoln, halter]",r2000/daily kos (d),"{'lincoln': 45.0, 'halter': 33.0}",lincoln,12.0,2010-04-15,<BLANK>,<BLANK>,0
6,florida senate,senate,FL,"[crist, meek]",quinnipiac,"{'crist': 48.0, 'meek': 34.0}",crist,14.0,2010-04-15,<BLANK>,<BLANK>,0
7,arizona senate,senate,AZ,"[mccain, hayworth]",rasmussen reports,"{'mccain': 47.0, 'hayworth': 42.0}",mccain,5.0,2010-04-16,<BLANK>,<BLANK>,0
8,nevada senate,senate,NV,"[lowden, reid]",lvrj/mason-dixon,"{'lowden': 47.0, 'reid': 37.0}",lowden,10.0,2010-04-16,<BLANK>,<BLANK>,0
9,new york senate,senate,NY,"[gillibrand, dioguardi]",siena,"{'gillibrand': 46.0, 'dioguardi': 27.0}",gillibrand,19.0,2010-04-18,<BLANK>,<BLANK>,0


In [4]:
df_g = nick[nick['race_name'].str.contains(r"[Gg]overnor")].reset_index(drop=True)
df_g

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
0,colorado governor,governor,CO,"[mcinnis, hickenlooper]",rasmussen reports,"{'hickenlooper': 42.0, 'mcinnis': 48.0}",mcinnis,6.0,2010-04-16,<BLANK>,<BLANK>,0
1,arizona governor,governor,AZ,"[brewer, mills]",rasmussen reports,"{'brewer': 26.0, 'mills': 18.0, 'martin': 12.0...",brewer,8.0,2010-04-17,<BLANK>,<BLANK>,0
2,new york governor,governor,NY,"[lazio, levy]",siena,"{'lazio': 29.0, 'levy': 15.0, 'paladino': 13.0}",lazio,14.0,2010-04-18,<BLANK>,<BLANK>,0
3,new york governor,governor,NY,"[cuomo, levy]",siena,"{'cuomo': 58.0, 'levy': 23.0}",cuomo,35.0,2010-04-18,<BLANK>,<BLANK>,0
4,new york governor,governor,NY,"[cuomo, lazio]",siena,"{'cuomo': 61.0, 'lazio': 24.0}",cuomo,37.0,2010-04-18,<BLANK>,<BLANK>,0
5,texas governor,governor,TX,"[perry, white]",rasmussen reports,"{'perry': 48.0, 'white': 44.0}",perry,4.0,2010-04-19,<BLANK>,<BLANK>,0
6,massachusetts governor,governor,MA,"[patrick, cahill]",western ne college,"{'patrick': 34.0, 'baker': 27.0, 'cahill': 29.0}",patrick,5.0,2010-04-19,<BLANK>,<BLANK>,0
7,florida governor,governor,FL,"[mccollum, sink]",quinnipiac,"{'mccollum': 40.0, 'sink': 36.0}",mccollum,4.0,2010-04-19,<BLANK>,<BLANK>,0
8,pennsylvania governor,governor,PA,"[corbett, wagner]",rasmussen reports,"{'corbett': 48.0, 'wagner': 27.0}",corbett,21.0,2010-04-20,<BLANK>,<BLANK>,0
9,pennsylvania governor,governor,PA,"[corbett, hoeffel]",rasmussen reports,"{'corbett': 49.0, 'hoeffel': 29.0}",corbett,20.0,2010-04-20,<BLANK>,<BLANK>,0


In [5]:
df_h = nick[nick['race_name'].str.contains(r"[Dd]istrict")].reset_index(drop=True)
df_h["district"] = df_h["race_name"].str.replace(r"^[a-z\. ]+ ","")
df_h["district"] = df_h["district"].str.replace(r"at.?large","0")
df_h["district"] = df_h["district"].str.replace(r"[^0-9]+$","")
df_h["district"] = df_h["district"].str.replace(r"^.* ","").astype("int64")
df_h

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
0,pennsylvania 12th district special election,house,PA,"[burns, critz]",ppp (d),"{'burns': 44.0, 'critz': 41.0}",burns,3.0,2010-04-20,<BLANK>,<BLANK>,12
1,new hampshire 1st district,house,NH,"[guinta, shea-porter]",ppp (d),"{'guinta': 46.0, 'shea-porter': 45.0}",guinta,1.0,2010-04-22,<BLANK>,<BLANK>,1
2,new hampshire 2nd district,house,NH,"[bass, swett]",ppp (d),"{'bass': 47.0, 'swett': 32.0}",bass,15.0,2010-04-22,<BLANK>,<BLANK>,2
3,new hampshire 1st district,house,NH,"[guinta, shea-porter]",wmur/unh,"{'guinta': 42.0, 'shea-porter': 38.0}",guinta,4.0,2010-04-30,<BLANK>,<BLANK>,1
4,new hampshire 2nd district,house,NH,"[bass, swett]",wmur/unh,"{'bass': 44.0, 'swett': 27.0}",bass,17.0,2010-04-30,<BLANK>,<BLANK>,2
5,maryland 1st district,house,MD,"[harris, kratovil]",pos (r),"{'harris': 39.0, 'kratovil': 36.0}",harris,3.0,2010-05-01,<BLANK>,<BLANK>,1
6,pennsylvania 12th district special election,house,PA,"[burns, critz]",r2000/daily kos (d),"{'burns': 46.0, 'critz': 40.0}",burns,6.0,2010-05-01,<BLANK>,<BLANK>,12
7,pennsylvania 12th district special election,house,PA,"[burns, critz]",r2000/daily kos (d),"{'critz': 40.0, 'burns': 46.0}",burns,6.0,2010-05-01,<BLANK>,<BLANK>,12
8,hawaii 1st district special election,house,HI,"[djou, case]",honolulu advertiser,"{'djou': 36.0, 'case': 28.0, 'hanabusa': 22.0}",djou,8.0,2010-05-02,<BLANK>,<BLANK>,1
9,pennsylvania 12th district special election,house,PA,"[critz, burns]",susquehanna,"{'critz': 44.0, 'burns': 38.0}",critz,6.0,2010-05-12,<BLANK>,<BLANK>,12


In [6]:
len(nick) == len(df_h) + len(df_g) + len(df_s)

True

In [7]:
house.head()

,date,party,name,chamber,state,district
0,2018-08-07,r,balderson,house,OH,12
1,2018-06-30,r,cloud,house,TX,27
2,2018-04-24,r,lesko,house,AZ,8
3,2018-03-13,d,lamb,house,PA,18
4,2017-11-07,r,curtis,house,UT,3


In [8]:
governor.head()

,date,party,name,chamber,state,end,district
0,2018-01-31,r,colyer,governor,KS,2019,0
1,2018-01-16,d,murphy,governor,NJ,2022,0
2,2018-01-13,d,northam,governor,VA,2022,0
3,2017-05-24,r,reynolds,governor,IA,2019,0
4,2017-04-10,r,ivey,governor,AL,2019,0


In [9]:
senate.head()

,date,party,name,chamber,state,district
0,2018-09-04,r,kyl,senate,AZ,0
1,2018-04-02,r,hydesmith,senate,MS,0
2,2018-01-03,d,jones,senate,AL,0
3,2018-01-03,d,smith,senate,MN,0
4,2017-02-09,r,strange,senate,AL,0


In [10]:
for i in reversed(range(len(governor))):
    m = governor.iloc[i,:]
    #matching names
    df_g.loc[(m["state"]==df_g["state"]) & \
    ([m['name'] in x.keys() for x in df_g["poll_results"]]) & \
    (m["date"] <= df_g["date"]) & \
    (df_g["date"] <= max(governor["date"])), "winner"] = m["name"]
    #matching parties
    df_g.loc[(m["state"]==df_g["state"]) & \
    ([m['name'] in x.keys() for x in df_g["poll_results"]]) & \
    (m["date"] <= df_g["date"]) & \
    (df_g["date"] <= max(governor["date"])), "winning_party"] = m["party"]
    
for i in range(len(governor)):
    m = governor.iloc[i,:]
    #matching names
    df_g.loc[(m["state"]==df_g["state"]) & \
    ([m['name'] in x.keys() for x in df_g["poll_results"]]) & \
    (m["date"] > df_g["date"]) & \
    (df_g["date"] <= max(governor["date"])), "winner"] = m["name"]
    #matching parties
    df_g.loc[(m["state"]==df_g["state"]) & \
    ([m['name'] in x.keys() for x in df_g["poll_results"]]) & \
    (m["date"] > df_g["date"]) & \
    (df_g["date"] <= max(governor["date"])), "winning_party"] = m["party"]
    

In [11]:
for i in reversed(range(len(senate))):
    m = senate.iloc[i,:]
    #matching names
    df_s.loc[(m["state"]==df_s["state"]) & \
    ([m['name'] in x.keys() for x in df_s["poll_results"]]) & \
    (m["date"] <= df_s["date"]) & \
    (df_s["date"] <= max(senate["date"])), "winner"] = m["name"]
    #matching parties
    df_s.loc[(m["state"]==df_s["state"]) & \
    ([m['name'] in x.keys() for x in df_s["poll_results"]]) & \
    (m["date"] <= df_s["date"]) & \
    (df_s["date"] <= max(senate["date"])), "winning_party"] = m["party"]

for i in range(len(senate)):
    m = senate.iloc[i,:]
    #matching names
    df_s.loc[(m["state"]==df_s["state"]) & \
    ([m['name'] in x.keys() for x in df_s["poll_results"]]) & \
    (m["date"] > df_s["date"]) & \
    (df_s["date"] <= max(senate["date"])), "winner"] = m["name"]
    #matching parties
    df_s.loc[(m["state"]==df_s["state"]) & \
    ([m['name'] in x.keys() for x in df_s["poll_results"]]) & \
    (m["date"] > df_s["date"]) & \
    (df_s["date"] <= max(senate["date"])), "winning_party"] = m["party"]
    

In [12]:
for i in reversed(range(len(house))):
    m = house.iloc[i,:]
    #matching names
    df_h.loc[(m["state"]==df_h["state"]) & \
    ([m['name'] in x.keys() for x in df_h["poll_results"]]) & \
    (m["date"] <= df_h["date"]) & \
    (m["district"]==df_h["district"]) & \
    (df_h["date"] <= max(house["date"])), "winner"] = m["name"]
    #matching parties
    df_h.loc[(m["state"]==df_h["state"]) & \
    ([m['name'] in x.keys() for x in df_h["poll_results"]]) & \
    (m["date"] <= df_h["date"]) & \
    (m["district"]==df_h["district"]) & \
    (df_h["date"] <= max(house["date"])), "winning_party"] = m["party"]

for i in range(len(house)):
    m = house.iloc[i,:]
    #matching names
    df_h.loc[(m["state"]==df_h["state"]) & \
    ([m['name'] in x.keys() for x in df_h["poll_results"]]) & \
    (m["date"] > df_h["date"]) & \
    (m["district"]==df_h["district"]) & \
    (df_h["date"] <= max(house["date"])), "winner"] = m["name"]
    #matching parties
    df_h.loc[(m["state"]==df_h["state"]) & \
    ([m['name'] in x.keys() for x in df_h["poll_results"]]) & \
    (m["date"] > df_h["date"]) & \
    (m["district"]==df_h["district"]) & \
    (df_h["date"] <= max(house["date"])), "winning_party"] = m["party"]

In [13]:
df_h.head()

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
0,pennsylvania 12th district special election,house,PA,"[burns, critz]",ppp (d),"{'burns': 44.0, 'critz': 41.0}",burns,3.0,2010-04-20,critz,d,12
1,new hampshire 1st district,house,NH,"[guinta, shea-porter]",ppp (d),"{'guinta': 46.0, 'shea-porter': 45.0}",guinta,1.0,2010-04-22,guinta,r,1
2,new hampshire 2nd district,house,NH,"[bass, swett]",ppp (d),"{'bass': 47.0, 'swett': 32.0}",bass,15.0,2010-04-22,bass,r,2
3,new hampshire 1st district,house,NH,"[guinta, shea-porter]",wmur/unh,"{'guinta': 42.0, 'shea-porter': 38.0}",guinta,4.0,2010-04-30,guinta,r,1
4,new hampshire 2nd district,house,NH,"[bass, swett]",wmur/unh,"{'bass': 44.0, 'swett': 27.0}",bass,17.0,2010-04-30,bass,r,2


In [14]:
df_h["winning_party"].unique()

array(['d', 'r', '<BLANK>'], dtype=object)

In [15]:
df_g.head()

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
0,colorado governor,governor,CO,"[mcinnis, hickenlooper]",rasmussen reports,"{'hickenlooper': 42.0, 'mcinnis': 48.0}",mcinnis,6.0,2010-04-16,hickenlooper,d,0
1,arizona governor,governor,AZ,"[brewer, mills]",rasmussen reports,"{'brewer': 26.0, 'mills': 18.0, 'martin': 12.0...",brewer,8.0,2010-04-17,brewer,r,0
2,new york governor,governor,NY,"[lazio, levy]",siena,"{'lazio': 29.0, 'levy': 15.0, 'paladino': 13.0}",lazio,14.0,2010-04-18,<BLANK>,<BLANK>,0
3,new york governor,governor,NY,"[cuomo, levy]",siena,"{'cuomo': 58.0, 'levy': 23.0}",cuomo,35.0,2010-04-18,cuomo,d,0
4,new york governor,governor,NY,"[cuomo, lazio]",siena,"{'cuomo': 61.0, 'lazio': 24.0}",cuomo,37.0,2010-04-18,cuomo,d,0


In [16]:
df_s.head()

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
0,florida senate,senate,FL,"[crist, rubio]",quinnipiac,"{'crist': 32.0, 'rubio': 30.0, 'meek': 24.0}",crist,2.0,2010-04-15,rubio,r,0
1,arkansas senate,senate,AR,"[boozman, lincoln]",r2000/daily kos (d),"{'boozman': 50.0, 'lincoln': 43.0}",boozman,7.0,2010-04-15,boozman,r,0
2,arkansas senate,senate,AR,"[boozman, halter]",r2000/daily kos (d),"{'boozman': 48.0, 'halter': 41.0}",boozman,7.0,2010-04-15,boozman,r,0
3,florida senate,senate,FL,"[rubio, meek]",quinnipiac,"{'rubio': 42.0, 'meek': 38.0}",rubio,4.0,2010-04-15,rubio,r,0
4,florida senate,senate,FL,"[rubio, crist]",quinnipiac,"{'rubio': 56.0, 'crist': 33.0}",rubio,23.0,2010-04-15,rubio,r,0


In [17]:
len(df_g[df_g["winner"]=="<BLANK>"])/len(df_g)

0.2548785344484269

In [18]:
df_g[df_g["winner"]=="<BLANK>"]

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
2,new york governor,governor,NY,"[lazio, levy]",siena,"{'lazio': 29.0, 'levy': 15.0, 'paladino': 13.0}",lazio,14.0,2010-04-18,<BLANK>,<BLANK>,0
7,florida governor,governor,FL,"[mccollum, sink]",quinnipiac,"{'mccollum': 40.0, 'sink': 36.0}",mccollum,4.0,2010-04-19,<BLANK>,<BLANK>,0
11,florida governor,governor,FL,"[mccollum, sink]",rasmussen reports,"{'mccollum': 45.0, 'sink': 38.0}",mccollum,7.0,2010-04-20,<BLANK>,<BLANK>,0
14,new hampshire governor,governor,NH,"[stephen, testerman]",ppp (d),"{'stephen': 29.0, 'testerman': 15.0, 'kimball'...",stephen,14.0,2010-04-21,<BLANK>,<BLANK>,0
15,california governor,governor,CA,"[whitman, poizner]",capitol weekly/probolsky (r),"{'whitman': 47.0, 'poizner': 19.0}",whitman,28.0,2010-04-22,<BLANK>,<BLANK>,0
16,california governor,governor,CA,"[whitman, poizner]",surveyusa,"{'whitman': 49.0, 'poizner': 27.0}",whitman,22.0,2010-04-22,<BLANK>,<BLANK>,0
18,wisconsin governor,governor,WI,"[neumann, barrett]",rasmussen reports,"{'neumann': 46.0, 'barrett': 46.0}",NaN,NaN,2010-04-23,<BLANK>,<BLANK>,0
21,maryland governor,governor,MD,"[o'malley, ehrlich]",rasmussen reports,"{'o'malley': 47.0, 'ehrlich': 44.0}",o'malley,3.0,2010-04-23,<BLANK>,<BLANK>,0
30,georgia governor,governor,GA,"[handel, barnes]",rasmussen reports,"{'barnes': 41.0, 'handel': 42.0}",handel,1.0,2010-04-28,<BLANK>,<BLANK>,0
31,georgia governor,governor,GA,"[oxendine, barnes]",rasmussen reports,"{'barnes': 43.0, 'oxendine': 45.0}",oxendine,2.0,2010-04-28,<BLANK>,<BLANK>,0


In [19]:
len(df_s[df_s["winner"]=="<BLANK>"])/len(df_s)

0.20057971014492754

In [20]:
df_s[df_s["winner"]=="<BLANK>"]

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
5,arkansas senate,senate,AR,"[lincoln, halter]",r2000/daily kos (d),"{'lincoln': 45.0, 'halter': 33.0}",lincoln,12.0,2010-04-15,<BLANK>,<BLANK>,0
6,florida senate,senate,FL,"[crist, meek]",quinnipiac,"{'crist': 48.0, 'meek': 34.0}",crist,14.0,2010-04-15,<BLANK>,<BLANK>,0
11,indiana senate,senate,IN,"[hostettler, ellsworth]",rasmussen reports,"{'hostettler': 50.0, 'ellsworth': 33.0}",hostettler,17.0,2010-04-19,<BLANK>,<BLANK>,0
17,wisconsin senate,senate,WI,"[feingold, wall]",rasmussen reports,"{'feingold': 49.0, 'wall': 43.0}",feingold,6.0,2010-04-22,<BLANK>,<BLANK>,0
18,wisconsin senate,senate,WI,"[feingold, westlake]",rasmussen reports,"{'feingold': 49.0, 'westlake': 38.0}",feingold,11.0,2010-04-22,<BLANK>,<BLANK>,0
19,california senate,senate,CA,"[campbell, fiorina]",surveyusa,"{'campbell': 34.0, 'fiorina': 27.0, 'devore': ...",campbell,7.0,2010-04-22,<BLANK>,<BLANK>,0
20,california senate,senate,CA,"[campbell, fiorina]",capitol weekly/probolsky (r),"{'campbell': 31.0, 'fiorina': 17.0, 'devore': ...",campbell,14.0,2010-04-22,<BLANK>,<BLANK>,0
27,north carolina senate,senate,NC,"[marshall, cunningham]",ppp (d),"{'marshall': 26.0, 'cunningham': 23.0, 'lewis'...",marshall,3.0,2010-04-27,<BLANK>,<BLANK>,0
28,north carolina senate,senate,NC,"[marshall, cunningham]",surveyusa,"{'marshall': 23.0, 'cunningham': 19.0, 'lewis'...",marshall,4.0,2010-04-27,<BLANK>,<BLANK>,0
29,arizona senate,senate,AZ,"[hayworth, glassman]",behavior research center,"{'hayworth': 37.0, 'glassman': 30.0}",hayworth,7.0,2010-04-28,<BLANK>,<BLANK>,0


In [21]:
len(df_h[df_h["winner"]=="<BLANK>"])/len(df_h)

0.3353028064992615

In [22]:
df_h[df_h["winner"]=="<BLANK>"]

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,district
31,washington 3rd district,house,WA,"[herrera, heck]",surveyusa,"{'herrera': 54.0, 'heck': 41.0}",herrera,13.0,2010-08-26,<BLANK>,<BLANK>,3
43,washington 3rd district,house,WA,"[herrera, heck]",surveyusa,"{'herrera': 52.0, 'heck': 43.0}",herrera,9.0,2010-09-16,<BLANK>,<BLANK>,3
54,new york 24th district,house,NY,"[arcuri, hanna]",siena,"{'arcuri': 48.0, 'hanna': 40.0}",arcuri,8.0,2010-09-21,<BLANK>,<BLANK>,24
132,new jersey 12th district,house,NJ,"[holt, sipprelle]",monmouth university,"{'holt': 51.0, 'sipprelle': 46.0}",holt,5.0,2010-10-13,<BLANK>,<BLANK>,12
137,washington 3rd district,house,WA,"[herrera, heck]","the hill/penn, schoen & berland (d)","{'herrera': 42.0, 'heck': 40.0}",herrera,2.0,2010-10-13,<BLANK>,<BLANK>,3
149,washington 3rd district,house,WA,"[herrera, heck]",surveyusa,"{'herrera': 53.0, 'heck': 42.0}",herrera,11.0,2010-10-14,<BLANK>,<BLANK>,3
201,massachusetts 6th district,house,MA,"[tisei, tierney]",boston globe,"{'tisei': 37.0, 'tierney': 31.0}",tisei,6.0,2012-10-01,<BLANK>,<BLANK>,6
253,new hampshire 1st district,house,NH,"[shea-porter, innis]",wmur/unh,"{'shea-porter': 43.0, 'innis': 32.0}",shea-porter,11.0,2013-10-18,<BLANK>,<BLANK>,1
256,new hampshire 1st district,house,NH,"[shea-porter, innis]",wmur/unh,"{'shea-porter': 43.0, 'innis': 33.0}",shea-porter,10.0,2014-02-04,<BLANK>,<BLANK>,1
262,new hampshire 1st district,house,NH,"[shea-porter, innis]",wmur/unh,"{'shea-porter': 45.0, 'innis': 29.0}",shea-porter,16.0,2014-04-17,<BLANK>,<BLANK>,1


In [23]:
max(senate["date"])

Timestamp('2018-09-04 00:00:00')

In [24]:
#df_h.to_pickle("HousePollwithWinner_Final.pkl")
#df_s.to_pickle("SenatePollwithWinner_Final.pkl")
#df_g.to_pickle("GovernorPollwithWinner_Final.pkl")